In [1]:
import pandas as pd
import sqlalchemy as sa
from redshift_credentials import database, username, password, host, port, schema
from sqlalchemy.orm import sessionmaker
import itertools
import pygsheets as pygs
from datetime import timedelta
from pandas.tseries.offsets import MonthEnd

In [2]:
def sql_con(in_username, in_password, in_host, in_port, in_database, in_schema):
    '''Create connection function'''
    connection = "redshift+psycopg2://%s:%s@%s:%s/%s" % (in_username, in_password, in_host, str(in_port), in_database)
    engine = sa.create_engine(connection)
    session = sessionmaker()
    session.configure(bind=engine)
    s = session()
    setpath = "SET search_path TO %s" % in_schema
    s.execute(setpath)
    return engine

In [3]:
g_drive_cred = r'C:\Users\darius kay\Dev\gitlab\test\python\credentials.json'
workbook = "https://docs.google.com/spreadsheets/d/1Y6D0-TRXMT3gwN3HITouUmMJrhUPqItTykXcCFfU23I/edit#gid=1763802414"

def clear_data_range(workbook_name, tab_name, data_range):
    start_end = str(data_range).split(':')
    start = start_end[0]
    print(start)
    end = start_end[1]
    print(end)
    print(g_drive_cred)
    # authorization
    gc = pygs.authorize(client_secret=g_drive_cred)
    # open the google spreadsheet
    ws = gc.open_by_url(workbook_name)
    # select the first sheet
    wks = ws.worksheet_by_title(tab_name)
    print(ws.id)
    wks.clear(start=start, end=end)
    return

def pd_to_gsheets(workbook_name, sheet_name, df, row_num, col_num):
    # authorization
    gc = pygs.authorize(client_secret=g_drive_cred)

    # open the google spreadsheet
    ws = gc.open_by_url(workbook_name)

    # select the first sheet
    wks = ws.worksheet_by_title(sheet_name)

    # update the designated sheet with df, starting at cell B2.
    wks.set_dataframe(df, (row_num, col_num))

    return

In [4]:
def date_checker(view, date, schedule):
    if view == 'forecast':
        if schedule == 'week':
            try:
                return date + timedelta(days=21)
            except TypeError:
                return
        elif schedule == 'month':
            try:
                return date + MonthEnd(1)
            except TypeError:
                return
    else:
        return date

In [5]:
# Define connection.
conn = sql_con(username, password, host, port, database, schema)

In [6]:
# Primary weekly forecast query.
weekly_forecast_query = "SELECT fulfillment_week_ended week_ended, %s, %s qty_returned, COUNT(*) qty_sold, %s * 1.0 / COUNT(*) rate FROM product_pull.ret_exch_rep_data_final GROUP BY 1%s HAVING DATE_PART('year', fulfillment_week_ended) = 2020"

# Actual weekly primary query.
weekly_actual_query = "SELECT fulfillment_week_ended week_ended, %s, %s qty_returned, COUNT(*) qty_sold, %s * 1.0 / COUNT(*) rate FROM product_pull.ret_exch_rep_data_final GROUP BY 1%s HAVING DATE_PART('year', fulfillment_week_ended) = 2020"

In [7]:
# Monthly actual primary query.
monthly_actual_query = "SELECT fulfillment_month_ended month_ended, %s, %s qty_returned, COUNT(*) qty_sold, %s * 1.0 / COUNT(*) rate FROM product_pull.ret_exch_rep_data_final GROUP BY 1%s HAVING DATE_PART('year', fulfillment_month_ended) = 2020"

# Primary monthly forecast query.
monthly_forecast_query = "SELECT fulfillment_month_ended month_ended, %s, %s qty_returned, COUNT(*) qty_sold, %s * 1.0 / COUNT(*) rate FROM product_pull.ret_exch_rep_data_final GROUP BY 1%s HAVING DATE_PART('year', fulfillment_month_ended) = 2020"

In [8]:
# Weekly Forecasted qty returned/exchanged query.
weekly_forecast_qty = "SUM(CASE WHEN send_back__type IN %s AND return_created_date <= fulfillment_week_ended+21 THEN 1 ELSE 0 END)"

# Monthly Forecasted qty returned/exchanged query.
monthly_forecast_qty = "SUM(CASE WHEN send_back__type IN %s AND return_created_date <= fulfillment_month_ended+21 THEN 1 ELSE 0 END)"

# Actual qty returned/exchanged query.
actual_qty = "SUM(CASE WHEN send_back__type in %s THEN 1 ELSE 0 END)"

In [9]:
product_categories = "CASE WHEN shape = 'Rectangle' AND size NOT ILIKE ('%9%x%12%') AND texture = 'Chenille' AND purpose != 'Outdoor' THEN 'rectangular_chenille' WHEN texture = 'Shag' THEN 'shag' WHEN shape = 'Round' THEN 'round' WHEN purpose = 'Outdoor' THEN 'outdoor' WHEN texture = 'Plush' THEN 'plush' WHEN size ILIKE ('%9%x%12%') THEN '9x12' WHEN texture = 'Rubber' AND product_sub_type = 'classic' THEN 'classic_rug_pad' WHEN texture = 'Rubber' AND product_sub_type = 'cushioned' THEN 'cushioned_rug_pad' END product"

# Edit for cushion only non-cover.
product_cushioned = "CASE WHEN product_sub_type = 'cushioned' THEN 'cushioned' WHEN product_sub_type = 'classic' THEN 'classic' END product"

In [10]:
query_params = {
    'send_back_product_weekly': {'view': {'actual': [weekly_actual_query, actual_qty], 'forecast': [weekly_forecast_query, weekly_forecast_qty]},
                              'product': {'all': "'all' product", 'categories': product_categories, 'cushioned': product_cushioned},
                              'send_back__type': {'send_back_both': "('return', 'exchange')",
                                                  'return': "('return')", 'exchange': "('exchange')"}},
    'send_back_product_monthly': {'view': {'actual': [monthly_actual_query, actual_qty], 'forecast': [monthly_forecast_query, monthly_forecast_qty]},
                          'product': {'all': "'all' product", 'categories': product_categories, 'cushioned': product_cushioned},
                          'send_back__type': {'send_back_both': "('return', 'exchange')",
                                              'return': "('return')", 'exchange': "('exchange')"}}
}

In [11]:
for key, value in query_params.items():
    '''Add logic to populate each tab in google sheet'''
#     print(key)
    tab_df = pd.DataFrame()
    combinations = []
    keys = value.keys()
    values = (value[key] for key in keys)
    combinations.append(dict(zip(keys, combination)) for combination in itertools.product(*values))
    for combination in combinations:
        for combo in combination:
            keys = list(keys)
            grouping = ""
            for i in range(len(keys)):
#                 print(query_params[key][keys[i]][combo[keys[i]]])
                if i == 0:
                    query = query_params[key][keys[i]][combo[keys[i]]][0]
#                     print(query)
                    calculation = query_params[key][keys[i]][combo[keys[i]]][1]
#                     print(calculation)
                    params = f"'{combo[keys[i]]}' AS {keys[i]}"
#                     print(params)
                else:
                    grouping = f"{grouping}, {i+1}"
                    if keys[i] == 'send_back__type':
                        calculation = calculation %(str(query_params[key][keys[i]][combo[keys[i]]]))
                        params = f"{params}, '{combo[keys[i]]}' {keys[i]}"
                    else:
                        params = f"{params}, {query_params[key][keys[i]][combo[keys[i]]]}"
            query = query %(params, calculation, calculation, grouping)
#             print(query)
#             print(calculation)
#             print(params)
#             print(grouping)
            df = pd.read_sql(query, conn)
            tab_df = tab_df.append(df)
#             tab_df = pd.concat([tab_df, df])
    
    if 'weekly' in key:
        # Add 21 days to week_ended for forecast only.
        tab_df['week_ended'] = tab_df.apply(lambda row: date_checker(row['view'], row['week_ended'], 'week'), axis=1)

        # Concat first four columns.
        keys.insert(0, 'week_ended')
        uid = tab_df[keys].astype(str).agg('_'.join, axis=1)
        tab_df.insert(loc=0, column='uid', value=uid)
    elif 'monthly' in key:
        # Add 21 days to month_ended for forecast only.
        tab_df['month_ended'] = tab_df.apply(lambda row: date_checker(row['view'], row['month_ended'], 'month'), axis=1)
        
        # Concat first four columns.
        keys.insert(0, 'month_ended')
        uid = tab_df[keys].astype(str).agg('_'.join, axis=1)
        tab_df.insert(loc=0, column='uid', value=uid)

    clear_data_range(workbook, key, "A1:J10000")
    pd_to_gsheets(workbook, key, tab_df, 1, 1)

A1
J10000
C:\Users\darius kay\Dev\gitlab\test\python\credentials.json
1Y6D0-TRXMT3gwN3HITouUmMJrhUPqItTykXcCFfU23I
A1
J10000
C:\Users\darius kay\Dev\gitlab\test\python\credentials.json
1Y6D0-TRXMT3gwN3HITouUmMJrhUPqItTykXcCFfU23I


In [ ]:
tab_df